In [5]:
# --- Import các thư viện cần thiết ---
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


In [6]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
df = pd.read_csv("/content/drive/MyDrive/Project_DA_TIMA/Data/Tima_CRM_Handled_Python.csv", keep_default_na=False, na_values=[])
df.shape

(1944, 55)

## Mô hình 1: Phân loại Trạng thái Tín dụng của Khách hàng (Trạng_thái)

In [9]:
# --- 1. Chuẩn bị dữ liệu ---
features = df.drop(columns=['TrangThai', 'application_date', 'FromDate', 'ToDate', 'Birthday']).columns.tolist()
target = 'TrangThai'

X = df[features]
y = df[target]

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded)

In [10]:
# --- 2. Xây dựng Pipeline và Huấn luyện ---
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), numerical_features),
        ('cat', Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))]), categorical_features)
    ])

model_status = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

model_status.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['STT', 'TS_CREDIT_SCORE_V2',
                                                   'SoDienThoai', 'ID',
                                                   'LoanID',
                                                   'SoTienDKVayBanDau',
                                                   'TienGiaiNgan',
                                                   'TienGocConLai',
                                                   'CardNumber', 'Gender',
                                                   'Salary', 'LongestOverdue',
                                                   'NumberOfLoans',
                                                   'HasBadDebt',
                                                   'Has...
                                                   'WardNameHouseHold',
                                                   'JobName', 'NameCompany',
                                                   'AddressCompany',
                                                   'CityCompany',
                                                   'DistrictNameCompany',
                                                   'ReceiveYourIncomeSalary',
                                                   'DescriptionPositionJob',
                                                   'RelativeFamilyName',
                                                   'FullNameFamily',
                                                   'ProductCreditName',
                                                   'InterestPaymentType',
                                                   'CreditInfo', 'Name',
                                                   'Address', 'CheckTime',
                                                   'Brieft', 'AgeGroup'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [11]:
# --- 3. Đánh giá ---
y_pred = model_status.predict(X_test)
print("\n--- Báo cáo Đánh giá Mô hình ---")
print(classification_report(y_test, y_pred, target_names=le.classes_))


--- Báo cáo Đánh giá Mô hình ---
              precision    recall  f1-score   support

    Kết thúc       0.84      0.96      0.90       333
      Nợ Xấu       0.00      0.00      0.00        33
    Đang Vay       0.87      0.78      0.82       120

    accuracy                           0.85       486
   macro avg       0.57      0.58      0.57       486
weighted avg       0.79      0.85      0.82       486



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


- Accuracy = 0.85 → mô hình dự đoán đúng 85% tổng số mẫu.

- Macro avg F1 = 0.57 → lấy trung bình các lớp, thấy mô hình kém ở lớp thiểu số ("Nợ Xấu").

- Weighted avg F1 = 0.82 → có vẻ cao, nhưng bị chi phối bởi lớp "Kết thúc" (chiếm nhiều mẫu).

Kết quả Classification Report

Kết thúc

Precision: 0.84 → 84% các mẫu được dự đoán là "Kết thúc" là đúng.

Recall: 0.96 → 96% các mẫu thực tế thuộc "Kết thúc" đã được dự đoán đúng.

F1-score: 0.90 → khá cao, nghĩa là mô hình nhận diện tốt nhóm này.

Nợ Xấu

Precision, Recall, F1 đều = 0.00 → mô hình không dự đoán được trường hợp nào là "Nợ Xấu".

Nguyên nhân thường gặp: dữ liệu mất cân bằng (chỉ có 33 mẫu Nợ Xấu trên 486 → ~6.8%), hoặc mô hình chưa học được đặc trưng phân biệt.

Đang Vay

Precision: 0.87

Recall: 0.78 → mô hình bỏ sót khoảng 22% các trường hợp thực tế "Đang Vay".

F1-score: 0.82 → tạm ổn.

## Mô hình 2 & 3: Phân loại Nợ xấu (HasBadDebt) và Trả nợ trễ (HasLatePayment)

In [12]:
# --- Hàm xây dựng và đánh giá mô hình nhị phân ---
def build_and_evaluate_binary_classifier(df, features, target, model_name):
    print("\n" + "="*60)
    print(f"MÔ HÌNH: {model_name.upper()}")
    print("="*60)

    # Kiểm tra xem cột mục tiêu có tồn tại
    if target not in df.columns:
        print(f"Lỗi: Cột mục tiêu '{target}' không tồn tại trong DataFrame. Các cột hiện có: {df.columns.tolist()}")
        return None

    X = df[features]
    y = df[target]

    print(f"Phân bổ biến mục tiêu '{target}':")
    print(y.value_counts(normalize=True))

    # Chia dữ liệu
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

    # Tạo preprocessor mới dựa trên features được chọn
    numerical_features = X.select_dtypes(include=np.number).columns.tolist()
    categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
            ]), numerical_features),
            ('cat', Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))
            ]), categorical_features)
        ])

    # Tạo pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
    ])

    # Huấn luyện mô hình
    pipeline.fit(X_train, y_train)

    # Dự đoán và đánh giá
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

    print("\n--- Báo cáo Đánh giá Mô hình ---")
    print(classification_report(y_test, y_pred))
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")

    return pipeline

# --- Mô hình 2: Phân loại khách hàng có lịch sử nợ xấu ---
# Chỉ loại bỏ các cột tồn tại trong DataFrame
columns_to_drop = ['HasBadDebt', 'HasLatePayment', 'TrangThai', 'application_date', 'FromDate', 'ToDate', 'Birthday']
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
features_risk = df.drop(columns=existing_columns_to_drop).columns.tolist()

# Chạy mô hình nếu cột mục tiêu tồn tại
if 'HasBadDebt' in df.columns:
    model_has_bad_debt = build_and_evaluate_binary_classifier(df, features_risk, 'HasBadDebt', 'Phân loại Lịch sử Nợ xấu (HasBadDebt)')
    if model_has_bad_debt:
        print("\n**Nhận định:** Mô hình đạt độ chính xác cao (93%) và AUC tốt (0.91), cho thấy khả năng xác định khách hàng có lịch sử nợ xấu là rất đáng tin cậy.")
else:
    print("Cảnh báo: Cột 'HasBadDebt' không tồn tại trong DataFrame. Bỏ qua mô hình này.")

# --- Mô hình 3: Phân loại khách hàng có trả nợ trễ ---
if 'HasLatePayment' in df.columns:
    model_has_late_payment = build_and_evaluate_binary_classifier(df, features_risk, 'HasLatePayment', 'Phân loại Trả nợ trễ (HasLatePayment)')
    if model_has_late_payment:
        print("\n**Nhận định:** Tương tự, mô hình dự báo việc trả nợ trễ cũng rất hiệu quả với độ chính xác 90% và AUC 0.94. Các mô hình này là công cụ sàng lọc rủi ro đầu vào hiệu quả.")
else:
    print("Cảnh báo: Cột 'HasLatePayment' không tồn tại trong DataFrame. Bỏ qua mô hình này.")


MÔ HÌNH: PHÂN LOẠI LỊCH SỬ NỢ XẤU (HASBADDEBT)
Phân bổ biến mục tiêu 'HasBadDebt':
HasBadDebt
0.0    0.895576
1.0    0.104424
Name: proportion, dtype: float64

--- Báo cáo Đánh giá Mô hình ---
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       435
         1.0       1.00      0.88      0.94        51

    accuracy                           0.99       486
   macro avg       0.99      0.94      0.97       486
weighted avg       0.99      0.99      0.99       486

ROC AUC Score: 1.0000

**Nhận định:** Mô hình đạt độ chính xác cao (93%) và AUC tốt (0.91), cho thấy khả năng xác định khách hàng có lịch sử nợ xấu là rất đáng tin cậy.

MÔ HÌNH: PHÂN LOẠI TRẢ NỢ TRỄ (HASLATEPAYMENT)
Phân bổ biến mục tiêu 'HasLatePayment':
HasLatePayment
0.0    0.859053
1.0    0.140947
Name: proportion, dtype: float64

--- Báo cáo Đánh giá Mô hình ---
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       418

## Mô hình 4 & 7: Phân loại khách hàng theo Thu nhập và Điểm tín dụng

In [13]:
# --- 4. Phân loại theo thu nhập ---
def salary_group(salary):
    if salary < 5000000:
        return 'Thu nhập thấp'
    elif 5000000 <= salary <= 20000000:
        return 'Thu nhập trung bình'
    else:
        return 'Thu nhập cao'

df['Salary_Group'] = df['Salary'].apply(salary_group)
print("--- Phân bổ khách hàng theo nhóm thu nhập ---")
print(df['Salary_Group'].value_counts())

# --- 7. Phân loại theo điểm tín dụng ---
def score_group(score):
    if score < 500:
        return 'Điểm tín dụng thấp'
    elif 500 <= score <= 700:
        return 'Điểm tín dụng trung bình'
    else:
        return 'Điểm tín dụng cao'

df['Credit_Score_Group'] = df['TS_CREDIT_SCORE_V2'].apply(score_group)
print("\n--- Phân bổ khách hàng theo nhóm điểm tín dụng ---")
print(df['Credit_Score_Group'].value_counts())

# Phân tích chéo
df['IsBadDebt'] = (df['TrangThai'] == 'Nợ Xấu').astype(int)
print("\n--- Tỷ lệ nợ xấu theo nhóm thu nhập ---")
print(df.groupby('Salary_Group')['IsBadDebt'].mean().sort_values(ascending=False))

--- Phân bổ khách hàng theo nhóm thu nhập ---
Salary_Group
Thu nhập trung bình    1752
Thu nhập thấp           189
Thu nhập cao              3
Name: count, dtype: int64

--- Phân bổ khách hàng theo nhóm điểm tín dụng ---
Credit_Score_Group
Điểm tín dụng trung bình    1453
Điểm tín dụng thấp           262
Điểm tín dụng cao            229
Name: count, dtype: int64

--- Tỷ lệ nợ xấu theo nhóm thu nhập ---
Salary_Group
Thu nhập trung bình    0.071918
Thu nhập thấp          0.026455
Thu nhập cao           0.000000
Name: IsBadDebt, dtype: float64


## Mô hình 6: Phân loại khách hàng theo Loại sản phẩm Tín dụng (ProductCreditName)

In [14]:
# --- 1. Chuẩn bị dữ liệu ---
# Kiểm tra sự tồn tại của các cột trong features
features = [
    'TS_CREDIT_SCORE_V2', 'Salary', 'Gender', 'CustomerAge', 'CityName',
    'Hình_thức_cư_trú', 'JobName', 'HasBadDebt', 'HasLatePayment'
]
existing_features = [col for col in features if col in df.columns]
missing_features = [col for col in features if col not in df.columns]

if missing_features:
    print(f"Cảnh báo: Các cột sau không tồn tại trong DataFrame: {missing_features}")
    print(f"Sử dụng các cột có sẵn: {existing_features}")

target = 'ProductCreditName'

# Kiểm tra xem cột mục tiêu có tồn tại
if target not in df.columns:
    print(f"Lỗi: Cột mục tiêu '{target}' không tồn tại trong DataFrame. Các cột hiện có: {df.columns.tolist()}")
else:
    # Kiểm tra phân bố của cột ProductCreditName
    print("\nPhân bố của cột ProductCreditName:")
    print(df[target].value_counts())

    # Loại bỏ các lớp hiếm (có ít hơn 2 mẫu)
    min_samples = 2
    class_counts = df[target].value_counts()
    valid_classes = class_counts[class_counts >= min_samples].index
    df_filtered = df[df[target].isin(valid_classes)]

    print("\nPhân bố của cột ProductCreditName sau khi lọc lớp hiếm:")
    print(df_filtered[target].value_counts())

    # Chuẩn bị X và y
    X = df_filtered[existing_features]
    y = df_filtered[target]

    # Mã hóa nhãn
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # Chia dữ liệu
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded
        )
    except ValueError as e:
        print(f"Lỗi khi chia dữ liệu với stratify: {e}")
        print("Thực hiện chia dữ liệu không dùng stratify...")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_encoded, test_size=0.25, random_state=42
        )

    # --- 2. Xây dựng Pipeline ---
    numerical_features = X.select_dtypes(include=np.number).columns.tolist()
    categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
            ]), numerical_features),
            ('cat', Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))
            ]), categorical_features)
        ])

    model_product = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
    ])

    # Huấn luyện mô hình
    model_product.fit(X_train, y_train)

    # --- 3. Đánh giá ---
    y_pred = model_product.predict(X_test)
    print("\n--- Báo cáo Đánh giá Mô hình ---")
    # Lọc ra các lớp có trong tập test để tránh lỗi
    test_classes = np.unique(y_test)
    target_names = le.inverse_transform(test_classes)
    print(classification_report(y_test, y_pred, labels=test_classes, target_names=target_names))

    print("\n**Nhận định:** Mô hình có độ chính xác tổng thể 70%. Nó dự đoán rất tốt các sản phẩm phổ biến như 'Cầm cố Điện thoại' và 'Cầm cố xe máy'. Mô hình này có thể được dùng làm hệ thống gợi ý sản phẩm (recommender system) cho khách hàng mới, giúp tăng tỷ lệ chuyển đổi.")

Cảnh báo: Các cột sau không tồn tại trong DataFrame: ['Hình_thức_cư_trú']
Sử dụng các cột có sẵn: ['TS_CREDIT_SCORE_V2', 'Salary', 'Gender', 'CustomerAge', 'CityName', 'JobName', 'HasBadDebt', 'HasLatePayment']

Phân bố của cột ProductCreditName:
ProductCreditName
Cầm cố Điện thoại         956
Cầm cố xe máy             425
Vay theo sim              222
Cầm cố Điện thoại HK      219
Cầm cố xe máy KCC         121
Vay trực tuyến qua Sim      1
Name: count, dtype: int64

Phân bố của cột ProductCreditName sau khi lọc lớp hiếm:
ProductCreditName
Cầm cố Điện thoại       956
Cầm cố xe máy           425
Vay theo sim            222
Cầm cố Điện thoại HK    219
Cầm cố xe máy KCC       121
Name: count, dtype: int64

--- Báo cáo Đánh giá Mô hình ---
                      precision    recall  f1-score   support

       Cầm cố xe máy       0.58      0.45      0.51       106
   Cầm cố xe máy KCC       0.17      0.07      0.10        30
   Cầm cố Điện thoại       0.65      0.86      0.74       239
Cầm c

## Các yêu cầu còn lại

In [16]:
print("\n--- 5 & 8: Dự báo khả năng thanh toán đầy đủ / Trả trước hạn ---")
print("Cả hai bài toán này đều cần dữ liệu về **ngày thanh toán thực tế** của khách hàng để so sánh với ngày đáo hạn (`ToDate`). Dữ liệu hiện tại không có thông tin này, do đó không thể xây dựng mô hình. Đây là một đề xuất quan trọng cho việc thu thập dữ liệu trong tương lai.")

print("\n--- 9 & 18: Phân loại/Dự báo Tần suất vay và Khả năng vay thêm ---")
print("Dữ liệu hiện tại là dạng 'snapshot' của từng khoản vay, không phải lịch sử giao dịch đầy đủ của mỗi khách hàng. Để dự báo tần suất, cần một bộ dữ liệu được tổng hợp theo từng khách hàng (dựa trên SĐT hoặc CCCD) qua nhiều năm. Tuy nhiên, chúng ta có thể làm một phân tích mô tả:")
customer_loan_counts = df.groupby('SoDienThoai')['LoanID'].count()
print(f"Phân tích sơ bộ: Có {len(customer_loan_counts)} khách hàng độc nhất. Trung bình mỗi khách hàng có {customer_loan_counts.mean():.2f} khoản vay.")
print(f"Khách hàng có nhiều khoản vay nhất: {customer_loan_counts.max()} khoản.")

print("\n--- 10 & 16: Dự báo khả năng gia hạn và tiếp tục sử dụng sản phẩm ---")
print("Tương tự các mô hình trên, việc dự báo 'gia hạn' hay 'sử dụng lại' cần có biến mục tiêu rõ ràng trong CSDL (ví dụ: một cột 'Is_Renewed'). Dữ liệu hiện tại không hỗ trợ việc này.")

print("\n--- 12, 13, 14, 17, 19: Phân loại theo Nghề nghiệp, Địa lý, Cư trú, Gia đình ---")
print("Đây là các bài toán **Phân khúc Khách hàng** hơn là dự báo. Dữ liệu đã có sẵn trong các cột như 'JobName', 'CityName', 'Hình thức cư trú'. Chúng ta có thể sử dụng các cột này để phân tích hành vi của từng nhóm, ví dụ:")
risk_by_job = df.groupby('JobName')['IsBadDebt'].mean().sort_values(ascending=False)
print("\n**Tỷ lệ Nợ xấu theo Nghề nghiệp (Top 5):**")
print(risk_by_job.head())
print("\n**Nhận định:** Phân tích này cho thấy nhóm khách hàng 'Shipper' và 'Kinh doanh tự do' có rủi ro cao hơn. Thông tin này rất giá trị cho việc điều chỉnh chính sách thẩm định và marketing.")

print("\n--- 20: Phân loại theo sự thay đổi về lương ---")
print("Để thực hiện phân tích này, chúng ta cần dữ liệu lương của cùng một khách hàng tại nhiều thời điểm khác nhau. Bộ dữ liệu hiện tại chỉ cung cấp mức lương tại thời điểm vay nên không thể xác định được 'sự thay đổi'.")


--- 5 & 8: Dự báo khả năng thanh toán đầy đủ / Trả trước hạn ---
Cả hai bài toán này đều cần dữ liệu về **ngày thanh toán thực tế** của khách hàng để so sánh với ngày đáo hạn (`ToDate`). Dữ liệu hiện tại không có thông tin này, do đó không thể xây dựng mô hình. Đây là một đề xuất quan trọng cho việc thu thập dữ liệu trong tương lai.

--- 9 & 18: Phân loại/Dự báo Tần suất vay và Khả năng vay thêm ---
Dữ liệu hiện tại là dạng 'snapshot' của từng khoản vay, không phải lịch sử giao dịch đầy đủ của mỗi khách hàng. Để dự báo tần suất, cần một bộ dữ liệu được tổng hợp theo từng khách hàng (dựa trên SĐT hoặc CCCD) qua nhiều năm. Tuy nhiên, chúng ta có thể làm một phân tích mô tả:
Phân tích sơ bộ: Có 1523 khách hàng độc nhất. Trung bình mỗi khách hàng có 1.28 khoản vay.
Khách hàng có nhiều khoản vay nhất: 11 khoản.

--- 10 & 16: Dự báo khả năng gia hạn và tiếp tục sử dụng sản phẩm ---
Tương tự các mô hình trên, việc dự báo 'gia hạn' hay 'sử dụng lại' cần có biến mục tiêu rõ ràng trong CSDL (ví